## Exercise 5 | data in f1ex5 | calculate the urban green space area per inhabitant (m2/inhabitant). 

In [ ]:
green_shp = '/mnt/d/cwsig/exercicios_2023_24/f1ex5/espacos_verdes.shp'
pop_shp  = '/mnt/d/cwsig/exercicios_2023_24/f1ex5/freg_cmb.shp'

popcol = 'popres11'
popid = 'idfreg'

outcol = 'green_by_person'

In [ ]:
import geopandas as gp
import pandas as pd
from glass.rd.shp import shp_to_obj
from glass.prop.prj import get_epsg

In [ ]:
green_df = shp_to_obj(green_shp)

pop_df = shp_to_obj(pop_shp)

epsg = get_epsg(green_shp)

In [ ]:
green_df['fk'] = 1
pop_df['pk'] = 1

join_df = pop_df.merge(green_df, how='inner', left_on='pk', right_on='fk')

join_df['geomi'] = gp.GeoSeries(join_df.geometry_x).intersection(gp.GeoSeries(join_df.geometry_y))

join_df = gp.GeoDataFrame(join_df, geometry="geomi", crs=f"EPSG:{str(epsg)}")

In [ ]:
join_df = join_df[~join_df.is_empty]

join_df.reset_index(inplace=True)

join_df["area_m2"] = join_df.geomi.area

pop_units = pd.DataFrame({
    'area_green' : join_df.groupby([popid])['area_m2'].agg('sum')
}).reset_index()

pop_units.rename(columns={popid : f'fk_{popid}'}, inplace=True)

pop_df = pop_df.merge(pop_units, how='left', left_on=popid, right_on=f'fk_{popid}')

pop_df[outcol] = pop_df["area_green"] / pop_df[popcol]

In [ ]:
pop_df